 # F710で操作しながらデータ取り
 別途 02, interactive_regressionを実行

### カメラの初期化

In [1]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

test


### racecarクラスのインスタンスを生成します。

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()
#デフォルトの上書き必須！
car.steering_gain   = 0.45
car.steering_offset = -0.1
car.throttle_gain   = 0.65
car.throttle_offset = 0.2 

### データセットのタスクの定義

In [3]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A','donkey']
#DATASETS = ['A','donkey']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

### データ収集(コントローラ操作用）

In [ ]:
#~ここはデフォルトのデータ取り~~~~~~~
import cv2
import numpy as np
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)       
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
camera_widget.on_msg(save_snapshot)
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])
display(data_collection_widget)

#~ここからｺﾝﾄﾛｰﾗ~~~~~~~
import multiprocessing
import time
import pygame
import math
import numpy as np
from pygame.locals import *
from Adafruit_GPIO import I2C

# pygameの初期化
pygame.init()
# ジョイスティックの初期化
pygame.joystick.init()
try:
    # ジョイスティックインスタンスの生成
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print("Joystick Name: " + joystick.get_name())
    print("Number of Button : " + str(joystick.get_numbuttons()))
    print("Number of Axis : " + str(joystick.get_numaxes()))
    print("Number of Hats : " + str(joystick.get_numhats()))
except pygame.error:
    print('ジョイスティックが接続されていません')

def expocurve(val):
    if val >0:
        return (math.exp(val)-1)/(math.e-1)
    else:
        return (math.exp(val*-1)-1)/(math.e-1)*-1
         
# F710の操作設定
# スティック
# 左スティック#０：左右、１：上下
# 右スティック#３：左右、４：上下
axis_steer = 0
axis_accel = 4
# ボタン
# 8: 'back', 9: 'start', 10: 'Logitech',
# 0: 'A', 1: 'B', 2: 'X', 3: 'Y',
# 4: 'LB',7: 'R1',8: 'right_stick_press',
button_A = 0
button_B = 1
button_Y = 3
button_X = 2
button_S = 7
button_R1 = 5
hat_rl = 0
hat_ud = 1

##スティックの指数関数変換
mode ="exp"

dirpath= "./road_following_A"

def save_snapshot_js2v(steering, throttle):
        x = steering *1 #マシン毎要調整
        y = throttle *-1 #マシン毎要調整
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  

        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        min, mid, max = int(224*0.2), int(224*0.5), int(224*0.8) #出力画像値の設定、マシン毎
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot = cv2.line(snapshot, (mid, min), (mid, max), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.line(snapshot, (min, mid), (max, mid), (255, 255, 255), thickness=1, lineType=cv2.LINE_4)
        snapshot = cv2.rectangle(snapshot, (min, min), (max, max), (0, 255, 0)) #マシン毎要調整 52,171, (112)
        snapshot = cv2.putText(snapshot, (str(x)+":"+str(y)), (x-50, y-20), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1, 8)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
def save_snapshot_js2v_noimage(steering, throttle,recording,cam):
        while recording.value:
            cam_re = np.array(cam[:],dtype='uint8').reshape(224,224,3)
            x = steering.value *1 #マシン毎要調整
            y = throttle.value *-1 #マシン毎要調整
            x = int(camera.width * (x / 2.0 + 0.5))
            y = int(camera.height * (y / 2.0 + 0.5))  
            # save to disk
            #dataset.save_entry_norefresh(path, camera.value, x, y)
            #dataset.save_entry(category_widget.value, camera.value, x, y)
            dataset.save_entry(category_widget.value, cam_re, x, y)
            #dataset.save_entry_norefresh(category_widget.value, cam_re, x, y)
            
            #time.sleep(0.2)
            print("x,y: ",x,y)
            print("data recorded")
            #count_widget.value = dataset.get_count(category_widget.value)
        
AI = False
running = True
recording = multiprocessing.Value('b', False) 
#False
steering = multiprocessing.Value('f', 0) # Value型の共有オブジェクト
throttle = multiprocessing.Value('f', 0) # Value型の共有オブジェクト

#cam = multiprocessing.Array('i', camera.value.flatten()) 
#↓ too slow, then use sharedctypes.RawArray
cam = multiprocessing.sharedctypes.RawArray('i', len(camera.value.flatten())) 
def update(change):
    np.asarray(cam)[:]=change['new'].flatten()
update({'new': camera.value})  # we call the function once to initialize
camera.observe(update, names='value')
camera.running = True

while running:
    #print(cam[0])
    for e in pygame.event.get():
        A =joystick.get_button(button_A)
        B =joystick.get_button(button_B)
        Y =joystick.get_button(button_Y)
        X =joystick.get_button(button_X)
        S =joystick.get_button(button_S)
        R1 =joystick.get_button(button_R1)
        Hud = joystick.get_hat(0)[hat_ud]
        if Hud != 0:
            car.throttle_gain = round(car.throttle_gain+(Hud*0.05),2)
            print("Current ThGain:",car.throttle_gain)
        if X:
            car.steering = expocurve(round(joystick.get_axis(axis_steer),2) )
            car.throttle = 0.3 *car.throttle_gain
            """
            camera.running = not(camera.running)
            if camera.running:
                print("Start camera/AI running...")
            else:
                print("Stop camera/AI running...")
            """
        elif mode == "exp":
            car.steering = expocurve(round(joystick.get_axis(axis_steer),2) )
            car.throttle = expocurve(round(joystick.get_axis(axis_accel),2) *-1 )

        else:
            car.steering = round(joystick.get_axis(axis_steer),2) 
            car.throttle = round(joystick.get_axis(axis_accel),2) *-1 

        if A : 
            #camera.running = False
            print("Breaking!")
            car.steering = 0
            car.throttle = -0.5
            time.sleep(0.05)            
            car.throttle = 0
            time.sleep(0.02)            
            car.throttle = -0.5
            time.sleep(0.02)            
            car.throttle = 0
            if recording.value:
                recording.value = not recording.value
                p1.terminate()
                #p1.join()
                print("Stop") 
                time.sleep(0.2)

        if recording.value :
            steering.value=car.steering
            throttle.value=car.throttle
        if Y:
            recording.value = not recording.value
            print("recording:",recording)
            if recording.value:
                p1 = multiprocessing.Process(target=save_snapshot_js2v_noimage,args=(steering,throttle,recording,cam,))
                #p1 = multiprocessing.Process(target=save_snapshot_js2v_noimage,args=(steering,throttle,recording,))
                p1.start()
                print("Start recording data!")
                time.sleep(0.2)
            else:
                p1.terminate()
                #p1.join()
                print("Stop") 
                time.sleep(0.2)
        if B:  
            save_snapshot_js2v(car.steering,car.throttle)
            print("Get running data!") 

        #f R1:  
        #   save_snapshot_js2v_noimage(car.steering,car.throttle)
        #   print("Get running data!") 

        if S:
            running = False
            camera.running = False
            print("Stop controlling...")



Joystick Name: Logitech Gamepad F710
Number of Button : 11
Number of Axis : 6
Number of Hats : 1
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
Breaking!
recording: <Synchronized wrapper for c_byte(1)>
Start recording data!
x,y:  59 112
data recorded
x,y:  113 112
data recorded
x,y:  160 112
data recorded
x,y:  87 112
data recorded
x,y:  112 112
data recorded
x,y:  146 112
data recorded
Get running data!
Get running data!
x,y:  156 
112data recorded
Get running data!
Get running data!
x,y:  156 112
data recorded
Get running data!
Get running data!
x,y:  222 112
data recorded
x,y:  224 112
data recorded
x,y:  114 112
data recorded
x,y:  112 112
data recorded
x,y:  112 112
data recorded
Get running data!
Get running data!
x,y:  114 112
Get running data!
Get running data!
data recorded
x,y:  75 112
data recorded
x,y:  154 112
data recorded
x,y:  215 112
data recorded
Get running data!
x,y:  78 112
data recorded
x,y:  172 112
data recorded
x,y:  112 112
data recorded
x,y:  49 112
data r

In [9]:
import numpy
cam_re = numpy.array(cam[:])
cam_re.reshape(224,224,3)

array([[[14, 12, 14],
        [22, 13, 24],
        [25, 14, 21],
        ...,
        [58, 59, 61],
        [63, 59, 55],
        [56, 39, 39]],

       [[20, 12, 19],
        [20,  7, 24],
        [25, 14, 21],
        ...,
        [51, 57, 59],
        [61, 60, 63],
        [64, 48, 42]],

       [[19, 12, 26],
        [19, 14, 24],
        [25, 12, 24],
        ...,
        [50, 59, 53],
        [58, 60, 60],
        [63, 51, 44]],

       ...,

       [[16, 13, 19],
        [20, 14, 16],
        [20, 14, 16],
        ...,
        [51, 51, 49],
        [48, 44, 40],
        [43, 35, 37]],

       [[ 0, 11, 10],
        [16, 13, 19],
        [25, 14, 21],
        ...,
        [46, 48, 48],
        [44, 38, 42],
        [41, 29, 35]],

       [[ 4, 10, 10],
        [10, 12, 22],
        [20, 14, 16],
        ...,
        [42, 46, 46],
        [42, 37, 36],
        [43, 30, 33]]])

### 転移学習用の学習済みモデルを読み込み

In [12]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])


display(model_widget)

### ライブにデータセット作成と学習を実行

In [5]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        category_index = dataset.categories.index(category_widget.value)
        x = output[2 * category_index]
        y = output[2 * category_index + 1]
        
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))
        
        prediction = image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

display(live_execution_widget)

In [6]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget)

NameError: name 'torch' is not defined

### 全部まとめて表示

以下のウィジェットは，複数クラスのx, yデータセットのラベル付けに使用できます． 1つの画像に対して，各クラスのインスタンスを1つだけラベリングすることをサポートしていますが（例：犬は1つだけ），1つの画像に対して複数のクラス（例：犬，猫，馬）をラベリングすることも可能です．

左上の画像をクリックすると、``category``の画像が、クリックした場所の``dataset``に保存されます。

| Widget | Description |
|--------|-------------|
| dataset | datasetを選択する |
| category | categoryを選択する |
| epochs | 転移学習のためのエポック数の数値を設定する |
| train | エポック数で指定された階数だけ選択されたデータセットを学習する  |
| evaluate | 選択したデータセットの1エポックあたりの精度を評価する |
| model path | モデルのパス名を指定 |
| load | model pathで指定した、学習済みモデルを読込 |
| save | model pathで指定した、学習済みモデルを保存 |
| stop | ライブデモを停止する |
| live | ライブデモを起動する |

In [15]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)

# カメラの終了処理(必須)

次のNotebookに進む際に必ずカメラの終了処理を実行してください。

In [65]:
import time
camera.running = False
time.sleep(1)
camera.cap.release()

In [ ]:
from multiprocessing import Process, Queue
import os
import time
import random


# Queueにデータを書き込む
def write(q):
    print('Process to write: {}'.format(os.getpid()))
    for value in ['A', 'B', 'C']:
        print('Put {} to queue...'.format(value))
        q.put(value)
        time.sleep(random.random())


# Queueからデータを読み取り
def read(q):
    print('Process to read: {}'.format(os.getpid()))
    while True:
        value = q.get(True)
        print('Get {} from queue.'.format(value))


# 親プロセスがQueueを作って、子プロセスに渡す
q = Queue()
#pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# pwを起動し、書き込み開始
pw.start()
# prを起動し、読み取り開始
pr.start()
# pwが終了するのを待つ
pw.join()
#time.sleep(3)
# prは無限ループなので、強制終了
pr.terminate()
